In [1]:
import pandas as pd
pd.set_option('display.max_columns', 50)
import ast
import os
import re

# import from ../utils.py
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
from utils import dwl_pdb_file,  get_resolution, get_dbref_data, get_gene_name, calculate_sbna_and_download, read_finalsum_decomp, read_finalsum, align_finalsum_with_uniprot

import paramiko # for ssh
import traceback

import seaborn  as sns
sns.set_style("darkgrid")
from matplotlib import pyplot as plt

c:\Users\liyoa\anaconda3\envs\sbna\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  "class": algorithms.Blowfish,


In [2]:
cgc_data = pd.read_csv("../data_prep/Census_all_with_pdb.csv")
cgc_data.head(3)

,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,PDB Structures,n_structures
0,A1CF,APOBEC1 complementation factor,29974.0,10:50799421-50885675,2,NaN,10q11.23,yes,NaN,melanoma,NaN,NaN,E,NaN,oncogene,Mis,NaN,NaN,NaN,"ACF,ACF64,ACF65,APOBEC1CF,ASP,CCDS73133.1,ENSG...",['2CPD'],1
1,ABI1,abl-interactor 1,10006.0,10:26746593-26860935,1,Yes,10p12.1,yes,NaN,AML,NaN,NaN,L,Dom,"TSG, fusion",T,KMT2A,NaN,NaN,"ABI-1,CCDS7150.1,E3B1,ENSG00000136754.17,NM_00...",['7LXE'],1
2,ABL1,v-abl Abelson murine leukemia viral oncogene h...,25.0,9:130713946-130885683,1,Yes,9q34.12,yes,NaN,"CML, ALL, T-ALL",NaN,NaN,L,Dom,"oncogene, fusion","T, Mis","BCR, ETV6, NUP214",NaN,NaN,"ABL,CCDS35165.1,ENSG00000097007.17,JTK7,NM_007...","['1AB2', '1AWO', '1BBZ', '1JU5', '1OPL', '1ZZP...",81


In [5]:
# if any contains "lung" or "NSCLC"
lung_genes = cgc_data[(cgc_data['Tumour Types(Somatic)'].str.contains("lung|NSCLC", case=False, na=False)) & (cgc_data['PDB Structures'].notnull())].reset_index(drop=True)
print("Number of genes with PDB structures associated with lung cancer: ", lung_genes.shape[0])
lung_genes.head(3)

Number of genes with PDB structures associated with lung cancer:  54


,Gene Symbol,Name,Entrez GeneId,Genome Location,Tier,Hallmark,Chr Band,Somatic,Germline,Tumour Types(Somatic),Tumour Types(Germline),Cancer Syndrome,Tissue Type,Molecular Genetics,Role in Cancer,Mutation Types,Translocation Partner,Other Germline Mut,Other Syndrome,Synonyms,PDB Structures,n_structures
0,AKT1,v-akt murine thymoma viral oncogene homolog 1,207.0,14:104769349-104794124,1,Yes,14q32.33,yes,NaN,"breast, colorectal, ovarian, NSCLC",NaN,NaN,E,Dom,oncogene,Mis,NaN,NaN,NaN,"AKT,CCDS9994.1,ENSG00000142208.15,P31749,PKB,P...","['1H10', '1UNP', '1UNQ', '1UNR', '2UVM', '2UZR...",36
1,ALK,anaplastic lymphoma kinase (Ki-1),238.0,2:29192774-29921566,1,Yes,2p23.2,yes,yes,"ALCL, NSCLC, neuroblastoma, inflammatory myofi...",neuroblastoma,familial neuroblastoma,"L, E, M",Dom,"oncogene, fusion","T, Mis, A","NPM1, TPM3, TFG, TPM4, ATIC, CLTC, MSN, RNF213...",NaN,NaN,"CCDS33172.1,CD246,ENSG00000171094.17,NM_004304...","['2KUP', '2KUQ', '2XB7', '2XBA', '2XP2', '2YFX...",77
2,ARAF,"A-Raf proto-oncogene, serine/threonine kinase",369.0,X:47561100-47571920,2,NaN,Xp11.3,yes,NaN,"cholangiocarcinoma, lung adenocarcinoma, Lange...",NaN,NaN,"E, L",NaN,oncogene,Mis,NaN,NaN,NaN,"ARAF1,CCDS75970.1,ENSG00000078061.12,NM_001256...","['1WXM', '2MSE']",2


### Download respective pdbs and read them

In [2]:
dwl_pdb_file("2OCJ")

True

In [25]:
pdbs = ['7SI1', "7UKV", "3POZ", "8F1X", "4LQM", "2ITZ", "6JWL", "5X27", "6GOD", "5UK9", \
        "6MBU", "4LDJ", "4L8G", "8AFB", "8AZX", "6GOF", "8TXG", "6GJ7", "6GOE", "8AZZ", \
            "7C40", "6MBQ", "4EPR", "3II5", "3Q4C", "3PRF", "3PPK", "3D4Q", "5FD2", "4FC0", \
                "4E26", "5C9C", "3IDP", "4MNF", "4G9R", '4LMN', '4U7Z', '3E8N', '3DY7', '3V04', '4U81', '4U80', '3PP1', '7SI5', '7SI4', '3JPX', '3JZG', '3K26', '3K27', '5U5H', '5H13', '5U5K', '1Y6A', '1Y6B', '3VHK', '3VHE', '3VID']

pdbs = ['6LVL', '6LVK', '2A91', '2QOO', '2QO2', '3FXX', '1UNR', '1UNQ', '2UZS', '1BG2', '2QOL', '2QO9', '2QOQ', '2QOI', '4J98', '4J96', '2GSF', '2UZR', '2QOF', '2UVM', '1MKJ', '2QOD', '2QOK', '3ETO', '2QOC', '2QO7', '5LT0', '3HMS']
pdbs = ['2YJR', '3AOX', '4CMT', '4CTB', '4FNW', '4FNX', '4FNY', ]

# all pdbs
# pdbs =
# for pdb in pdbs:
for pdb in set(t):
    print(dwl_pdb_file(pdb))

# for pdb in [i[0] for i in t]:
#     print(dwl_pdb_file(pdb))

True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [6]:
# count number of structures for all gene
n_total = lung_genes['n_structures'].sum()

for _, row in lung_genes.iterrows():
    gene = row['Gene Symbol']
    pdb_structures = ast.literal_eval(row['PDB Structures'])
    for pdb_id in pdb_structures:
        n_total -= 1
        print(f"Downloading PDB file {pdb_id} for {gene} ({n_total} structures remaining)", end='\r')
        # check if the file already exists
        if not os.path.exists(f'pdb_files/{pdb_id}.pdb'):
            if not dwl_pdb_file(pdb_id): # returns True or False if the file was downloaded
                continue

KeyboardInterrupt: 

In [ ]:
pdb_data = []
uniprot_map = {}

# count number of structures for all gene
n_total = lung_genes['n_structures'].sum()

for _, row in lung_genes.iterrows():
    gene = row['Gene Symbol']
    pdb_structures = ast.literal_eval(row['PDB Structures'])
    for pdb_id in pdb_structures:
        n_total -= 1
        # check if the file already exists
        if not os.path.exists(f'../pdb_files/{pdb_id}.pdb'):
            continue
        # read the file
        resolution = get_resolution(pdb_id)
        dbref = get_dbref_data(pdb_id)
        for i in dbref:
            chain = i['chain']
            start = i['start']
            end = i['end']
            uniprot = i['uniprot']
            if uniprot not in uniprot_map:
                uniprot_map[uniprot] = get_gene_name(uniprot)
            assoc_gene = uniprot_map[uniprot]
            pdb_data.append([gene, pdb_id, resolution, chain, start, end, uniprot, assoc_gene])

pdb_data = pd.DataFrame(pdb_data, columns=['gene', 'pdb_id', 'resolution', 'chain', 'seq_start', 'seq_end', 'uniprot_id', 'assoc_gene'])
pdb_data.head()
pdb_data['length'] = pdb_data['seq_end'] - pdb_data['seq_start'] + 1

In [ ]:
# sort gene ascending, length descending and resolution ascneding
pdb_data.sort_values(by=['gene', 'length', 'resolution', "pdb_id"], ascending=[True, False, True, True]).to_csv("best_pdbs.csv", index=False)

#  Calculate SBNA

In [3]:
lung_genes = pd.read_excel('lung_genes.xlsx', sheet_name="final")

# groupby pdb and concatenate the chain in a list
tmp = lung_genes.groupby('PDB')['Chain'].apply(list).reset_index()
# lung_genes = lung_genes[:23]

In [19]:
for pdb in set(t):
    print(get_dbref_data(pdb))

[{'chain': 'A', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'B', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'C', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'D', 'uniprot': 'P04637', 'start': 94, 'end': 293}]
[{'chain': 'A', 'uniprot': 'P04637', 'start': 94, 'end': 312}, {'chain': 'B', 'uniprot': 'P04637', 'start': 94, 'end': 312}, {'chain': 'C', 'uniprot': 'P04637', 'start': 94, 'end': 312}, {'chain': 'D', 'uniprot': 'P04637', 'start': 94, 'end': 312}]
[{'chain': 'A', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'B', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'C', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'D', 'uniprot': 'P04637', 'start': 94, 'end': 293}]
[{'chain': 'A', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'B', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'C', 'uniprot': 'P04637', 'start': 94, 'end': 293}, {'chain': 'D', 'uniprot': 'P04637', 'start': 94, 'end': 293}]
[{'c

In [26]:
pdbs

['2YJR', '3AOX', '4CMT', '4CTB', '4FNW', '4FNX', '4FNY']

In [3]:
host = "m3-dtn.massive.org.au"
username = "yliy0004"
with open("../pw.txt", "r") as f:
    password = f.read().strip()

client = paramiko.client.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
client.connect(host, username=username, password=password)

# check if R module is loaded (uses .bashrc - not the best solution because STRUDEL cannot be used, make sure to correct this if using STRUDEL)
stdin,stdout,stderr=client.exec_command('R --version') 
print(stdout.read().decode())
print(stderr.read().decode())

# test the connection
# stdin,stdout,stderr=client.exec_command('ls')
# print(stdout.read().decode())
# print(stderr.read().decode())

sftp_client=client.open_sftp()


# ==================== SBNA ====================
# for _, row in tmp.iterrows():
#     pdb_id = row['PDB']
#     chains = row['Chain']
# for pdb_id in pdbs:
#     try:
#         calculate_sbna_and_download(client, sftp_client, pdb_id) #chains=chains)
#     except Exception as e:
#         print(f"Failed to process {pdb_id} with error:")
#         print(traceback.print_exc())
#         continue

# for pdb, chains in pdb_chain_dict.items():
# for pdb in pdbs:
for pdb in ['7DHY']:
    try:
        calculate_sbna_and_download(client, sftp_client, pdb), #chains=chains)
    except Exception as e:
        print(f"Failed to process {pdb} with error:")
        print(traceback.print_exc())
        continue

# close the connection
client.close()
sftp_client.close()

R version 4.0.5 (2021-03-31) -- "Shake and Throw"
Copyright (C) 2021 The R Foundation for Statistical Computing
Platform: x86_64-pc-linux-gnu (64-bit)

R is free software and comes with ABSOLUTELY NO WARRANTY.
You are welcome to redistribute it under the terms of the
GNU General Public License versions 2 or 3.
For more information about these matters see
https://www.gnu.org/licenses/.




Processing 7DHY with chain A...
Starting phenix.pdbtools
on Sun Jun  2 13:22:47 2024 by yliy0004

Processing files:
-------------------------------------------------------------------------------

  Found model, 7DHY.pdb

Processing PHIL parameters:
-------------------------------------------------------------------------------

  Adding command-line PHIL:
  -------------------------
    keep=Chain A

Final processed PHIL parameters:
-------------------------------------------------------------------------------
  data_manager {
    model {
      file = "7DHY.pdb"
    }
    default_model = "7DHY.pdb"


In [46]:
# read lung_pdb file
lung_pdbs = pd.read_csv("lung genes.csv")
lung_pdbs

,gene,pdb_id,resolution,chain,seq_start,seq_end,uniprot_id,assoc_gene,length,mutant,mutations,pair,notes
0,EGFR,7SI1,1.60,A,695,1022,P00533,EGFR,328,N,NaN,NaN,NaN
1,EGFR,7UKV,2.40,A,695,1022,P00533,EGFR,328,N,NaN,NaN,NaN
2,EGFR,3POZ,1.50,A,696,1022,P00533,EGFR,327,N,NaN,NaN,NaN
3,EGFR,8F1X,2.30,A,695,1022,P00533,EGFR,328,N,NaN,NaN,NaN
4,EGFR,4LQM,2.50,A,694,1022,P00533,EGFR,329,Y,L858R,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,ALK,4CTB,1.79,A,1093,1411,Q9UM73,ALK,319,N,NaN,NaN,NaN
255,ALK,2YJR,1.90,A,1093,1411,Q9UM73,ALK,319,Y,F1174L,NaN,NaN
256,ALK,7R7R,1.94,A,1093,1411,Q9UM73,ALK,319,N,NaN,NaN,NaN
257,ALK,5IMX,2.12,A,1093,1411,Q9UM73,ALK,319,N,NaN,NaN,NaN


In [34]:
# # check if sbna results were downloaded
print("Failed to calculate SBNA for:")
for _, row in lung_pdbs.iterrows():
    pdb_id = row['pdb_id']
    chains = row['chain']
    for chain in chains:
    # check if the file already exists
        if not os.path.exists(f'../sbna_results/{pdb_id}/{chain}'):
            # if not dwl_pdb_file(pdb_id): # returns True or False if the file was downloaded
            print(row['gene'], pdb_id, chain)


Failed to calculate SBNA for:
EGFR 4LQM A
ERBB4 2AHX A
ERBB4 2AHX B
ERBB4 2R4B A
ERBB4 2R4B B
DDR2 6FER A
DDR2 6FER B
DDR2 6FER C
DDR2 6FER D
DDR2 6FER E
DDR2 6FER F
DDR2 6FER G
DDR2 6FER H
DDR2 6FER I
DDR2 6FER J
DDR2 6FER K
DDR2 6FER L
PTPRT 2OOQ A
PTPRT 2OOQ B


### aligning for one pdb only (is the next section but singular)

In [6]:
# read lung_pdb file
lung_pdbs = pd.read_csv("lung genes.csv")
lung_pdbs.head(2)

,gene,pdb_id,resolution,chain,seq_start,seq_end,uniprot_id,assoc_gene,length,mutant,mutations,pair,notes
0,EGFR,7SI1,1.6,A,695,1022,P00533,EGFR,328,N,NaN,NaN,NaN
1,EGFR,7UKV,2.4,A,695,1022,P00533,EGFR,328,N,NaN,NaN,NaN


In [4]:
pdb_id = '7DHY'

for chain in [x['chain'] for x in get_dbref_data(pdb_id)]:
    print(chain)
    print(read_finalsum(pdb_id, chain).head(3))
    print(read_finalsum_decomp(pdb_id, chain).head(3))
# read_finalsum(pdb_id, chain).merge(read_finalsum_decomp(pdb_id, chain), on=['num', 'res'], how='inner').drop('res_code_x', axis=1)

A
   res res_code  num  network_score
0  SER        S   96      -1.126763
1  VAL        V   97       0.411958
2  PRO        P   98       2.463157
     Acid  SecondOrderIntermodularDegree_AVERAGE  \
0  CYS229                               0.161653   
1  TRP146                              -0.083144   
2  ARG273                               0.125345   

   NodeEdgeBetweennessSTRIDE_sidechain_MAX  LigandMULTIMERCENTROIDSC_MIN  num  \
0                                 1.143199                     -1.015263  229   
1                                 0.727006                     -1.779925  146   
2                                 0.339070                     -0.482529  273   

   res res_code  
0  CYS        C  
1  TRP        W  
2  ARG        R  
B
   res res_code  num  network_score
0  SER        S   96      -1.126763
1  VAL        V   97       0.206682
2  PRO        P   98       2.042096
     Acid  SecondOrderIntermodularDegree_AVERAGE  \
0  GLU271                               0.295379  

In [7]:
ns_data = pd.DataFrame()

for _, row in lung_pdbs[lung_pdbs['pdb_id']==pdb_id].iterrows():
    pdb_id = row['pdb_id']
    chain = row['chain']
    # print(pdb_id, chain)
    try:
        tmp = read_finalsum(pdb_id, chain).merge(read_finalsum_decomp(pdb_id, chain), on=['num', 'res'], how='inner').drop('res_code_x', axis=1)
    except:
        continue
    # rename columns
    tmp = tmp.rename(columns={"res_code_y": "res_code"})
    tmp = align_finalsum_with_uniprot(tmp, pdb_id, chain)
    # tmp = tmp[tmp['num']==12].reset_index(drop=True)
    tmp['pdb_id'] = pdb_id
    tmp['chain'] = chain
    tmp['gene'] = row['gene']
    tmp['mutant'] = row['mutant']
    tmp['mutations'] = row['mutations']
    ns_data = pd.concat([ns_data, tmp])

# rename columns
ns_data = ns_data.rename(columns={"res": "sbna_aa3", "res_code": "sbna_aa1", "num": "sbna_num", "uniprot_res": "uniprot_aa"})
ns_data = ns_data[["gene", "pdb_id", "chain", "sbna_aa3", "sbna_num", "sbna_aa1", "uniprot_num", "uniprot_aa", "network_score", "SecondOrderIntermodularDegree_AVERAGE", "NodeEdgeBetweennessSTRIDE_sidechain_MAX", "LigandMULTIMERCENTROIDSC_MIN", 'mutant', 'mutations']]

print(len(ns_data))
ns_data.head(5)

784


,gene,pdb_id,chain,sbna_aa3,sbna_num,sbna_aa1,uniprot_num,uniprot_aa,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,mutant,mutations
0,TP53,7DHY,A,SER,96,S,94,S,-1.126763,-0.768789,-0.417127,-0.059153,Y,G245S
1,TP53,7DHY,A,VAL,97,V,97,V,0.411958,-0.573712,0.448125,-0.537545,Y,G245S
2,TP53,7DHY,A,PRO,98,P,98,P,2.463157,-0.170044,1.897883,-0.735318,Y,G245S
3,TP53,7DHY,A,SER,99,S,99,S,0.848752,-0.720570,-0.347014,-1.916336,Y,G245S
4,TP53,7DHY,A,GLN,100,Q,100,Q,0.547100,-0.353719,0.184136,-0.716683,Y,G245S


In [10]:
aa_properties = pd.read_csv("../data_prep/aa_properties.csv")
# make dictionary of aa1:Acid Type
aa_properties = dict(zip(aa_properties['aa1'], aa_properties['Acid Type']))
aa_properties

{'I': 'Hydrophobic',
 'A': 'Hydrophobic',
 'F': 'Hydrophobic',
 'L': 'Hydrophobic',
 'M': 'Hydrophobic',
 'P': 'Special cases',
 'V': 'Hydrophobic',
 'W': 'Hydrophobic',
 'C': 'Special cases',
 'G': 'Special cases',
 'T': 'Polar uncharged',
 'S': 'Polar uncharged',
 'Y': 'Hydrophobic',
 'Q': 'Polar uncharged',
 'N': 'Polar uncharged',
 'H': 'Polar charged positive',
 'K': 'Polar charged positive',
 'E': 'Polar charged negative',
 'R': 'Polar charged positive',
 'D': 'Polar charged negative'}

In [12]:
ns_data['cgc_tier'] = 1
ns_data['cancer_role'] = "TSG"
ns_data["uniprot_aa_type"] = ns_data['uniprot_aa'].map(aa_properties)
ns_data["sbna_aa_type"] = ns_data['sbna_aa1'].map(aa_properties)
ns_data

,gene,pdb_id,chain,sbna_aa3,sbna_num,sbna_aa1,uniprot_num,uniprot_aa,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,mutant,mutations,cgc_tier,cancer_role,uniprot_aa_type,sbna_aa_type
0,TP53,7DHY,A,SER,96,S,94,S,-1.126763,-0.768789,-0.417127,-0.059153,Y,G245S,1,TSG,Polar uncharged,Polar uncharged
1,TP53,7DHY,A,VAL,97,V,97,V,0.411958,-0.573712,0.448125,-0.537545,Y,G245S,1,TSG,Hydrophobic,Hydrophobic
2,TP53,7DHY,A,PRO,98,P,98,P,2.463157,-0.170044,1.897883,-0.735318,Y,G245S,1,TSG,Special cases,Special cases
3,TP53,7DHY,A,SER,99,S,99,S,0.848752,-0.720570,-0.347014,-1.916336,Y,G245S,1,TSG,Polar uncharged,Polar uncharged
4,TP53,7DHY,A,GLN,100,Q,100,Q,0.547100,-0.353719,0.184136,-0.716683,Y,G245S,1,TSG,Polar uncharged,Polar uncharged
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,TP53,7DHY,D,GLU,287,E,287,E,-0.052733,-1.013234,-0.466253,-1.426754,Y,G245S,1,TSG,Polar charged negative,Polar charged negative
192,TP53,7DHY,D,ASN,288,N,288,N,-1.089911,-1.169358,-0.441454,-0.520901,Y,G245S,1,TSG,Polar uncharged,Polar uncharged
193,TP53,7DHY,D,LEU,289,L,289,L,-0.687241,-0.748215,-0.118174,-0.179147,Y,G245S,1,TSG,Hydrophobic,Hydrophobic
194,TP53,7DHY,D,ARG,290,R,290,R,-4.354151,-1.169271,-0.466253,2.718628,Y,G245S,1,TSG,Polar charged positive,Polar charged positive


In [13]:
ns_data.to_csv("tmp.csv", index=False)

# merge final_sum with cosmic data (adding cgc_tier and cancer_role)

In [2]:
# read lung_pdb file
lung_pdbs = pd.read_csv("lung genes.csv")
lung_pdbs.head(2)

,gene,pdb_id,resolution,chain,seq_start,seq_end,uniprot_id,assoc_gene,length,mutant,mutations,pair,notes
0,EGFR,7SI1,1.6,A,695,1022,P00533,EGFR,328,N,NaN,NaN,NaN
1,EGFR,7UKV,2.4,A,695,1022,P00533,EGFR,328,N,NaN,NaN,NaN


In [3]:
# first add  cgc_tier and cancer_role
cgc_data = pd.read_csv("../data_prep/Census_allTue Apr  9 04_58_38 2024.csv", usecols=['Gene Symbol', 'Tier', 'Role in Cancer'])
cgc_data.head(2)

,Gene Symbol,Tier,Role in Cancer
0,A1CF,2,oncogene
1,ABI1,1,"TSG, fusion"


In [5]:
ns_data = pd.DataFrame()

for _, row in lung_pdbs.iterrows():
    pdb_id = row['pdb_id']
    chain = row['chain']
    # print(pdb_id, chain)
    try:
        tmp = read_finalsum(pdb_id, chain).merge(read_finalsum_decomp(pdb_id, chain), on=['num', 'res'], how='inner').drop('res_code_x', axis=1)
    except:
        continue
    # rename columns
    tmp = tmp.rename(columns={"res_code_y": "res_code"})
    tmp = align_finalsum_with_uniprot(tmp, pdb_id, chain)
    # tmp = tmp[tmp['num']==12].reset_index(drop=True)
    tmp['pdb_id'] = pdb_id
    tmp['chain'] = chain
    tmp['gene'] = row['gene']
    tmp['mutant'] = row['mutant']
    tmp['mutations'] = row['mutations']
    ns_data = pd.concat([ns_data, tmp])

# rename columns
ns_data = ns_data.rename(columns={"res": "sbna_aa3", "res_code": "sbna_aa1", "num": "sbna_num", "uniprot_res": "uniprot_aa"})
ns_data = ns_data[["gene", "pdb_id", "chain", "sbna_aa3", "sbna_num", "sbna_aa1", "uniprot_num", "uniprot_aa", "network_score", "SecondOrderIntermodularDegree_AVERAGE", "NodeEdgeBetweennessSTRIDE_sidechain_MAX", "LigandMULTIMERCENTROIDSC_MIN", 'mutant', 'mutations']]

print(len(ns_data))
ns_data.head(5)

FinalSum file not found for 4LQM
FinalSum file not found for 2AHX
FinalSum file not found for 2AHX
FinalSum file not found for 2R4B
FinalSum file not found for 2R4B
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 6FER
FinalSum file not found for 2OOQ
FinalSum file not found for 2OOQ
60305


,gene,pdb_id,chain,sbna_aa3,sbna_num,sbna_aa1,uniprot_num,uniprot_aa,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,mutant,mutations
0,EGFR,7SI1,A,SER,695,S,695,S,-1.946806,-0.964584,-0.368011,0.614212,N,NaN
1,EGFR,7SI1,A,GLY,696,G,696,G,-1.901057,-0.900271,-0.530286,0.470500,N,NaN
2,EGFR,7SI1,A,GLU,697,E,697,E,-0.544335,-0.288050,-0.341759,-0.085474,N,NaN
3,EGFR,7SI1,A,ALA,698,A,698,A,-2.229062,-1.095856,-0.596766,0.536440,N,NaN
4,EGFR,7SI1,A,PRO,699,P,699,P,-2.403619,-1.158033,-0.596766,0.648820,N,NaN


In [6]:
# add cgc_tier and cancer_role from cgc_data
cgc_data = pd.read_csv("../data_prep/Census_all_with_pdb.csv", usecols=['Gene Symbol', 'Tier', 'Role in Cancer'])

# rename columns
cgc_data = cgc_data.rename(columns={"Gene Symbol": "gene", "Tier": "cgc_tier", "Role in Cancer": "cancer_role"})

# merge with tmp2
ns_data = ns_data.merge(cgc_data, left_on='gene', right_on='gene', how='left')
ns_data.head()

,gene,pdb_id,chain,sbna_aa3,sbna_num,sbna_aa1,uniprot_num,uniprot_aa,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,mutant,mutations,cgc_tier,cancer_role
0,EGFR,7SI1,A,SER,695,S,695,S,-1.946806,-0.964584,-0.368011,0.614212,N,NaN,1,oncogene
1,EGFR,7SI1,A,GLY,696,G,696,G,-1.901057,-0.900271,-0.530286,0.470500,N,NaN,1,oncogene
2,EGFR,7SI1,A,GLU,697,E,697,E,-0.544335,-0.288050,-0.341759,-0.085474,N,NaN,1,oncogene
3,EGFR,7SI1,A,ALA,698,A,698,A,-2.229062,-1.095856,-0.596766,0.536440,N,NaN,1,oncogene
4,EGFR,7SI1,A,PRO,699,P,699,P,-2.403619,-1.158033,-0.596766,0.648820,N,NaN,1,oncogene


In [3]:
# ns_data.to_csv("lung_genes_sbna.csv", index=False)

ns_data = pd.read_csv("lung_genes_sbna.csv")
print(len(ns_data))

56754


In [130]:
# # change cancer_role for 
# # TP53 to "TSG", NOTCH1 to "oncogene", ERBB4 to "oncogene"
# ns_data.loc[ns_data['gene']=="TP53", "cancer_role"] = "TSG"
# ns_data.loc[ns_data['gene']=="NOTCH1", "cancer_role"] = "oncogene"
# ns_data.loc[ns_data['gene']=="ERBB4", "cancer_role"] = "oncogene"

# # change "oncogene, fusion" to "oncogene"
# ns_data.loc[ns_data['cancer_role']=="oncogene, fusion", "cancer_role"] = "oncogene"

### Aggreate data by position. (only for wild types)

In [4]:
# Filter for wild types
ns_data = ns_data[ns_data['mutant']=="N"]

# aggregate data by gene, uniprot_num, uniprot_aa, sbna_aa1
ns_data = ns_data[ns_data['uniprot_num']!="?"]
ns_data['uniprot_num'] = ns_data['uniprot_num'].astype(int)
ns_data = ns_data.groupby(['gene', 'uniprot_num', 'uniprot_aa', 'sbna_aa1']).agg(\
    {'network_score': 'mean', 'SecondOrderIntermodularDegree_AVERAGE': 'mean', 'NodeEdgeBetweennessSTRIDE_sidechain_MAX': 'mean', \
     'LigandMULTIMERCENTROIDSC_MIN': 'mean', 'cgc_tier': 'first', 'cancer_role': 'first'}).reset_index()
ns_data.head()

,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role
0,AKT1,1,M,M,-1.442056,-0.831746,-0.478642,0.131669,1,oncogene
1,AKT1,2,S,S,-1.028050,-0.712966,-0.406600,-0.091516,1,oncogene
2,AKT1,3,D,D,-0.337630,-0.602712,-0.611381,-0.876463,1,oncogene
3,AKT1,4,V,V,0.369376,-0.335788,-0.067975,-0.773139,1,oncogene
4,AKT1,5,A,A,0.258724,-0.365234,-0.080011,-0.703969,1,oncogene


In [5]:
# ns_data.to_csv("wt_ns_aggregated.csv", index=False)
ns_data.to_csv("ns_aggregated_all.csv", index=False)

In [138]:
ns_data = pd.read_csv("ns_aggregated_all.csv")

# find rows where there are more than one row for each uniprot_aa
diff_aas = ns_data[ns_data.duplicated(subset=['gene', 'uniprot_num'], keep=False)].sort_values(['gene', 'uniprot_num'], ascending=True).reset_index(drop=True)
diff_aas['mutant'] = diff_aas['uniprot_aa']!=diff_aas['sbna_aa1']
diff_aas.to_csv("diff_aas.csv", index=False)
print(len(diff_aas))
diff_aas.head()

# merged_data[merged_data['sbna_aa1'] != merged_data['uniprot_aa']].sort_values(['gene', 'uniprot_num'], ascending=True)

72


,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role,mutant
0,AKT1,17,E,E,0.217918,-0.232960,-0.532687,-0.983564,1,oncogene,False
1,AKT1,17,E,K,0.178303,-0.516073,-0.260502,-0.954878,1,oncogene,True
2,ALK,1097,C,C,-2.128042,-0.866995,-0.384675,0.876372,1,oncogene,False
3,ALK,1097,C,S,-2.087775,-0.949926,-0.454795,0.683053,1,oncogene,True
4,ALK,1174,F,F,2.649683,0.714982,1.521564,-0.413137,1,oncogene,False


### try to calculate network scores again for all comparison of WT vs mutant 
* (i did a mistake before of taking NS of all structures even like mutant vs mutant)
* note there is no distinction between chains here for comparisons

In [14]:
mut_data = pd.read_csv("lung_genes_sbna.csv")
mut_data = mut_data[mut_data['uniprot_num'] != "?"]
mut_data['uniprot_num'] = mut_data['uniprot_num'].astype(int)
mut_data = mut_data.groupby(['gene', 'mutations', 'uniprot_num', 'uniprot_aa', 'sbna_aa1'])['network_score'].mean().reset_index()
mut_data

,gene,mutations,uniprot_num,uniprot_aa,sbna_aa1,network_score
0,AKT1,E17K,1,M,M,-0.160771
1,AKT1,E17K,2,S,S,0.357676
2,AKT1,E17K,3,D,D,-0.140439
3,AKT1,E17K,4,V,V,0.953059
4,AKT1,E17K,5,A,A,0.181981
...,...,...,...,...,...,...
6298,TP53,V272M,285,E,E,1.349385
6299,TP53,V272M,286,E,E,-0.337362
6300,TP53,V272M,287,E,E,-2.176509
6301,TP53,V272M,288,N,N,-2.209905


In [15]:
# read wt_ns_aggregated.csv
wt_data = pd.read_csv("wt_ns_aggregated.csv")
wt_data = wt_data[wt_data['uniprot_num'] != "?"]
wt_data['uniprot_num'] = wt_data['uniprot_num'].astype(int)
wt_data

,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role,cbioportal_freq,uniprot_aa_type
0,AKT1,1,M,M,-1.442056,-0.831746,-0.478642,0.131669,1,oncogene,0,Nonpolar
1,AKT1,63,M,M,-0.798672,-0.378048,-0.040428,0.380196,1,oncogene,0,Nonpolar
2,AKT1,118,M,M,-1.451314,-0.850082,-0.685275,-0.084043,1,oncogene,0,Nonpolar
3,ALK,1089,M,M,-0.358116,1.118639,-0.459551,1.017204,1,oncogene,0,Nonpolar
4,ALK,1166,M,M,-0.979742,-0.512846,-0.413243,0.053652,1,oncogene,0,Nonpolar
...,...,...,...,...,...,...,...,...,...,...,...,...
10749,TP53,238,C,C,3.678561,1.766149,1.912412,0.000000,1,TSG,57,Polar
10750,TP53,242,C,C,0.418538,0.852515,-0.433977,0.000000,1,TSG,77,Polar
10751,TP53,275,C,C,2.052943,1.229223,0.823720,0.000000,1,TSG,66,Polar
10752,TP53,277,C,C,-0.779711,-0.483881,-0.295830,0.000000,1,TSG,45,Polar


In [16]:
# merge with mut_data
merged_data = wt_data.merge(mut_data, on=['gene', 'uniprot_num', 'uniprot_aa', 'sbna_aa1'], how='inner', suffixes=('_wt', '_mut'))
# fill nan in cancer_role with unknown
merged_data['cancer_role'] = merged_data['cancer_role'].fillna("unknown")
merged_data = merged_data.sort_values(['gene', 'uniprot_num'])
merged_data

,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score_wt,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role,cbioportal_freq,uniprot_aa_type,mutations,network_score_mut
0,AKT1,1,M,M,-1.442056,-0.831746,-0.478642,0.131669,1,oncogene,0,Nonpolar,E17K,-0.160771
208,AKT1,2,S,S,-1.028050,-0.712966,-0.406600,-0.091516,1,oncogene,0,Polar,E17K,0.357676
649,AKT1,3,D,D,-0.337630,-0.602712,-0.611381,-0.876463,1,oncogene,1,Charged,E17K,-0.140439
990,AKT1,4,V,V,0.369376,-0.335788,-0.067975,-0.773139,1,oncogene,0,Nonpolar,E17K,0.953059
1436,AKT1,5,A,A,0.258724,-0.365234,-0.080011,-0.703969,1,oncogene,0,Nonpolar,E17K,0.181981
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3813,TP53,289,L,L,-1.299322,-0.861989,-0.437332,0.000000,1,TSG,1,Nonpolar,N239Y,-1.167156
3814,TP53,289,L,L,-1.299322,-0.861989,-0.437332,0.000000,1,TSG,1,Nonpolar,"R273C, T284R",-3.219438
3815,TP53,289,L,L,-1.299322,-0.861989,-0.437332,0.000000,1,TSG,1,Nonpolar,R273H,0.671923
3816,TP53,289,L,L,-1.299322,-0.861989,-0.437332,0.000000,1,TSG,1,Nonpolar,R280K,-0.849960


In [17]:
merged_data['diff_ns'] = merged_data['network_score_mut'] - merged_data['network_score_wt']

* note: cbioportal_freq here means the total number of mutations at that residue. not how many times this particular mutation has happened

In [18]:
merged_data.to_csv("sbna_wt_vs_mut.csv", index=False)

# adding TCGA data
* Using wild type structures only 

In [139]:
ns_data = pd.read_csv("wt_ns_aggregated.csv")

ns_data = ns_data[ns_data['uniprot_num']!="?"].reset_index(drop=True)
ns_data['uniprot_num'] = ns_data['uniprot_num'].astype(int)
ns_data

,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role
0,AKT1,1,M,M,-1.442056,-0.831746,-0.478642,0.131669,1,oncogene
1,AKT1,2,S,S,-1.028050,-0.712966,-0.406600,-0.091516,1,oncogene
2,AKT1,3,D,D,-0.337630,-0.602712,-0.611381,-0.876463,1,oncogene
3,AKT1,4,V,V,0.369376,-0.335788,-0.067975,-0.773139,1,oncogene
4,AKT1,5,A,A,0.258724,-0.365234,-0.080011,-0.703969,1,oncogene
...,...,...,...,...,...,...,...,...,...,...
10829,TPR,138,L,L,1.147110,0.583953,0.932649,0.369492,1,fusion
10830,TPR,139,E,E,-1.110253,-0.500517,-0.218686,0.391051,1,fusion
10831,TPR,140,Y,Y,-1.315210,-0.560337,-0.245537,0.509335,1,fusion
10832,TPR,141,L,L,-0.579938,0.123206,-0.154097,0.549048,1,fusion


## 1. Match mutations in residue data only!

In [140]:
def extract_position(mutation_position):
    try:
        chromosome, position_range = mutation_position.split(':')
        start_position, end_position = position_range.split('-')
        # Verify that the start and end positions are the same
        if start_position == end_position:
            return int(start_position)
        else:
            return None
    except:
        # if mutation_position is float then return it
        if type(mutation_position) == float:
            return mutation_position
        return None

In [141]:
# read cmc data
cmc_data = pd.read_csv('../CMC/cmc_export.tsv', sep='\t', usecols=[
    'GENE_NAME', 'AA_MUT_START', 'AA_MUT_STOP', 'AA_WT_ALLELE_SEQ', 'AA_MUT_ALLELE_SEQ', 'Mutation AA', 'Mutation Description AA', \
        'MUTATION_SIGNIFICANCE_TIER', 'GENOMIC_WT_ALLELE_SEQ', 'GENOMIC_MUT_ALLELE_SEQ', "Mutation genome position GRCh37", "Mutation genome position GRCh38"])

cmc_data = cmc_data[cmc_data['AA_MUT_START'] == cmc_data['AA_MUT_STOP']]

# mutation not nan in AA_WT_ALLELE_SEQ	Mutation AA	Mutation Description AA
cmc_data = cmc_data.dropna(subset=['AA_WT_ALLELE_SEQ', 'Mutation AA'])

# only use missense mutations
cmc_data = cmc_data[cmc_data['Mutation Description AA'] == 'Substitution - Missense']

# remove rows with null AA_WT_ALLELE_SEQ
cmc_data = cmc_data[cmc_data['AA_WT_ALLELE_SEQ'].notnull()] 
    
cmc_data['GRCh38 position'] = cmc_data['Mutation genome position GRCh38'].apply(extract_position)
cmc_data = cmc_data[cmc_data['GRCh38 position'].notnull()]
cmc_data['GRCh38 position'] = cmc_data['GRCh38 position'].astype(int)

# filter genes that are in lung_genes
cmc_data = cmc_data[cmc_data['GENE_NAME'].isin(ns_data['gene'])]

print(len(cmc_data))
cmc_data.head(3)

21583


,GENE_NAME,Mutation AA,AA_MUT_START,AA_MUT_STOP,GENOMIC_WT_ALLELE_SEQ,GENOMIC_MUT_ALLELE_SEQ,AA_WT_ALLELE_SEQ,AA_MUT_ALLELE_SEQ,Mutation Description AA,Mutation genome position GRCh37,Mutation genome position GRCh38,MUTATION_SIGNIFICANCE_TIER,GRCh38 position
68794,ERBB4,p.R842L,842,842,C,A,R,L,Substitution - Missense,2:212295788-212295788,2:211431063-211431063,Other,211431063
68795,ERBB4,p.E810Q,810,810,C,G,E,Q,Substitution - Missense,2:212426687-212426687,2:211561962-211561962,Other,211561962
68796,ERBB4,p.L567F,567,567,G,A,L,F,Substitution - Missense,2:212537906-212537906,2:211673181-211673181,Other,211673181


In [142]:
# find rows where chromosome position match to more than one uniprot position
is_one_to_one = cmc_data.groupby(['GENE_NAME', 'GRCh38 position'])['AA_MUT_START'].nunique() == 1
is_one_to_one = is_one_to_one.reset_index()
tmp = is_one_to_one[is_one_to_one["AA_MUT_START"]==False]['GENE_NAME'].unique()

# assert none of them are in lung genes (otherwise would need to do some manual mapping)
assert len(set(tmp).intersection(set(ns_data['gene'].unique()))) == 0

In [143]:
# now match chromosome position to uniprot num
# for each gene_name, aa_mut_start create set of chromosome positions
chromosome_positions = cmc_data.groupby(['GENE_NAME', 'AA_MUT_START'])['GRCh38 position'].apply(set).reset_index()
chromosome_positions.head()

,GENE_NAME,AA_MUT_START,GRCh38 position
0,AKT1,3,{104792637}
1,AKT1,4,"{104792633, 104792634}"
2,AKT1,11,{104792611}
3,AKT1,14,"{104792602, 104792603}"
4,AKT1,15,{104792600}


Note: this assumes that CMC data is as large as TCGA...as in every mutation in TCGA is also in CMC...will need to check and manually add after if no match

In [144]:
ns_data = ns_data.merge(chromosome_positions, left_on=['gene', 'uniprot_num'], right_on=['GENE_NAME', 'AA_MUT_START'], how='left')
ns_data.head(2)

,gene,uniprot_num,uniprot_aa,sbna_aa1,network_score,SecondOrderIntermodularDegree_AVERAGE,NodeEdgeBetweennessSTRIDE_sidechain_MAX,LigandMULTIMERCENTROIDSC_MIN,cgc_tier,cancer_role,GENE_NAME,AA_MUT_START,GRCh38 position
0,AKT1,1,M,M,-1.442056,-0.831746,-0.478642,0.131669,1,oncogene,NaN,NaN,NaN
1,AKT1,2,S,S,-1.028050,-0.712966,-0.406600,-0.091516,1,oncogene,NaN,NaN,NaN


In [160]:
def tcga_get_chr_position(dna_change):
    # example "chr12:g.25245350C>A" to 25245350
    match = re.search(r'chr\d+:g\.(\d+)', dna_change)
    if match:
        return match.group(1)
    else:
        return None

# read all TCGA files ending with .tsv
tcga_data = pd.DataFrame()
for file in os.listdir("../TCGA"):
    if file.endswith(".tsv"):
        tmp = pd.read_csv(f"../TCGA/{file}", sep='\t')
        gene = file.split('_')[0] # assume the file name is written properly...
        tmp['gene'] = gene
        tmp = tmp.rename(columns={f'num_{gene}_cases': 'num_cases_in_gene'}) # rename num_<GENE>_cases to num_cases
        tcga_data = pd.concat([tcga_data, tmp])

# missense only
tcga_data = tcga_data[(tcga_data['type']=="Substitution") & (tcga_data['consequence']=="Missense")] 

# extract position
tcga_data['GRCh38_pos'] = tcga_data['dna_change'].apply(tcga_get_chr_position).astype(int)

tcga_data = tcga_data.reset_index(drop=True)

tcga_data[['gene', 'wt_aa', 'res_pos', 'mut_aa']] = tcga_data['protein_change'].str.extract(r'(\w+) (\w)(\d+)(\w)')
tcga_data.head()

,ssm_id,dna_change,protein_change,type,consequence,num_ssm_affected_cases,num_cases_in_gene,ssm_affected_cases_percentage,num_gdc_ssm_affected_cases,num_gdc_ssm_cases,gdc_ssm_affected_cases_percentage,vep_impact,sift_impact,sift_score,polyphen_impact,polyphen_score,gene,GRCh38_pos,wt_aa,res_pos,mut_aa
0,ab96fb54-dfc0-58d9-b727-20a857d58dad,chr14:g.104780214C>T,AKT1 E17K,Substitution,Missense,69,4230,1.63,69,16477,0.42,MODERATE,deleterious,0.00,probably_damaging,1.000,AKT1,104780214,E,17,K
1,5bedd0f4-b0e1-503a-b857-a2f54f3bdc6c,chr14:g.104780145C>T,AKT1 E40K,Substitution,Missense,5,4230,0.12,5,16477,0.03,MODERATE,tolerated,0.16,possibly_damaging,0.515,AKT1,104780145,E,40,K
2,85931b10-2dfc-5711-ad10-52e8df9d4116,chr14:g.104776708A>G,AKT1 W80R,Substitution,Missense,4,4230,0.09,4,16477,0.02,MODERATE,deleterious,0.01,probably_damaging,0.930,AKT1,104776708,W,80,R
3,95594822-4344-5194-912d-99baa0a765b1,chr14:g.104780108A>C,AKT1 L52R,Substitution,Missense,4,4230,0.09,4,16477,0.02,MODERATE,deleterious,0.00,probably_damaging,0.993,AKT1,104780108,L,52,R
4,79df6966-11ae-5d3b-ab71-36da48e404f3,chr14:g.104776711G>T,AKT1 Q79K,Substitution,Missense,2,4230,0.05,2,16477,0.01,MODERATE,deleterious,0.02,probably_damaging,0.967,AKT1,104776711,Q,79,K


In [161]:
# assert that there are unique chromosome positions per gene
assert (tcga_data.groupby(['gene', 'GRCh38_pos'])['res_pos'].nunique() == 1).all()

In [162]:
# save tcga_data
tcga_data.to_csv("../TCGA/lung_genes_all_muts_tcga.csv", index=False)

In [147]:
counts = []
for _, row in ns_data.iterrows():
    gene = row['gene']
    # find corresponding row in tcga_data
    
    # position is nan
    if pd.isnull(row['GRCh38 position']):
        counts.append(0)
        continue

    # find corresponding rows in tcga_data
    tmp = tcga_data[(tcga_data['gene']==gene) & (tcga_data['GRCh38_pos'].isin(row['GRCh38 position'])) & (tcga_data['wt_aa'] == row['uniprot_aa'])]
    
    # SUM num_gdc_ssm_affected_cases BY POSITION
    counts.append(tmp['num_gdc_ssm_affected_cases'].sum()) 

ns_data['tcga_num_cases'] = counts

In [148]:
# also fill missing values in cancer_role
ns_data['cancer_role'] = ns_data['cancer_role'].fillna('Unknown')

In [149]:
ns_data.to_csv("wt_ns_aggregated.csv", index=False)